Extraindo informações do Bureau Balance: 
* https://www.kaggle.com/c/home-credit-default-risk/discussion/58445
* https://www.kaggle.com/c/home-credit-default-risk/discussion/57750

In [2]:
import pandas as pd
import numpy as np
import data
import charts
import pickle
import seaborn as sns

pd.set_option('display.max_columns', 500)
%matplotlib inline

In [2]:
def find_missing(data):
    # number of missing values
    count_missing = data.isnull().sum().values
    # total records
    total = data.shape[0]
    # percentage of missing
    ratio_missing = count_missing / total
    # return a dataframe to show: feature name, # of missing and % of missing
    return pd.DataFrame(data={'missing_count':count_missing, 'missing_ratio':ratio_missing}, index=data.columns.values)

In [5]:
application_train_df = pd.read_csv('dataset/application_train.csv')
application_train_df.head()

KeyboardInterrupt: 

## Bureau features

In [79]:
# Montagem da tabela relacionada ao bureau
bureau_full_df = pd.read_csv('dataset/bureau.csv')
#bureau_balance_df = pd.read_csv('dataset/bureau_balance.csv')
#bureau_full_df = bureau_df.join(bureau_balance_df.set_index('SK_ID_BUREAU'), on='SK_ID_BUREAU')

# Adicionando o target para facilitar analises futuras e treinamento do meu futuro modelo
#bureau_full_df = bureau_full_df.join(application_train_df.set_index('SK_ID_CURR')['TARGET'], on='SK_ID_CURR')
bureau_full_df.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [39]:
bureau_full_df['CREDIT_TYPE'].unique()

array(['Consumer credit', 'Credit card', 'Mortgage', 'Car loan',
       'Microloan', 'Loan for working capital replenishment',
       'Loan for business development', 'Real estate loan'], dtype=object)

In [18]:
with open('dataset/bureau_full.pkl', 'wb') as handle:
    model_predicts = pickle.dump(bureau_full_df, handle)

### NOTAS
> * Numeros negativos em datas (Ex.: DAYS_CREDIT, DAYS_CREDIT_ENDDATE etc.) significam que e no passado e positivos no futuro

### TODO:
> Ao verificar o mesmo cliente, podemos identificar que o mesmo pode estar registrado em diversos "escritórios" diferentes. 
> Isso pode gerar uma série de features interessantes:
> * OK: Quantos créditos já foram pedidos por este cliente?
> * OK: Gerar features baseados no numero de itens para cada cliente no CREDIT_ACTIVE
> * OK: Qual o tempo entre um emprestimo e outro?
> * OK: Qual o tempo médio entre um emprestimo e outro?
> * OK: Numero total de dias do emprestimo e tempo médio de emprestimos realizados
> * Existem dois ou mais pedidos de empréstimo ao mesmo tempo?
> * Existem créditos atrasados (CREDIT_DAY_OVERDUE)?
>   * Quantos creditos estao atrasados?
> * Quantos creditos ativos e nao ativos de cada tipo (CREDIT_TYPE) foram abertos?
> * Qual a porcentagem ja foi paga do credito (DAYS_CREDIT_ENDDATE e DAYS_CREDIT depois normalizar)
> * Qual a taxa de inadimplencia por SK_ID_BUREAU?

#### Quantos créditos já foram pedidos por este cliente?

In [128]:
bureau_full_df[(bureau_full_df.CREDIT_ACTIVE == 'Closed') & (bureau_full_df.DAYS_CREDIT_ENDDATE > 0)]

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,TARGET,DAYS_CREDIT_DIFF,DAYS_CREDIT_DIFF_MEAN
353,389599,5714529,Closed,currency 1,-672,0,10285.0,-531.0,NaN,0,135000.0,0.0,0.0,0.0,Credit card,-429,NaN,0.0,0,139.285714
354,389599,5714529,Closed,currency 1,-672,0,10285.0,-531.0,NaN,0,135000.0,0.0,0.0,0.0,Credit card,-429,NaN,0.0,343,139.285714
355,389599,5714529,Closed,currency 1,-672,0,10285.0,-531.0,NaN,0,135000.0,0.0,0.0,0.0,Credit card,-429,NaN,0.0,28,139.285714
356,389599,5714529,Closed,currency 1,-672,0,10285.0,-531.0,NaN,0,135000.0,0.0,0.0,0.0,Credit card,-429,NaN,0.0,131,139.285714
357,389599,5714529,Closed,currency 1,-672,0,10285.0,-531.0,NaN,0,135000.0,0.0,0.0,0.0,Credit card,-429,NaN,0.0,299,139.285714
358,389599,5714529,Closed,currency 1,-672,0,10285.0,-531.0,NaN,0,135000.0,0.0,0.0,0.0,Credit card,-429,NaN,0.0,123,139.285714
359,389599,5714529,Closed,currency 1,-672,0,10285.0,-531.0,NaN,0,135000.0,0.0,0.0,0.0,Credit card,-429,NaN,0.0,51,139.285714
360,389599,5714530,Closed,currency 1,-1304,0,1253.0,-1287.0,NaN,0,225000.0,0.0,0.0,0.0,Consumer credit,-429,NaN,0.0,0,139.285714
361,389599,5714530,Closed,currency 1,-1304,0,1253.0,-1287.0,NaN,0,225000.0,0.0,0.0,0.0,Consumer credit,-429,NaN,0.0,343,139.285714
362,389599,5714530,Closed,currency 1,-1304,0,1253.0,-1287.0,NaN,0,225000.0,0.0,0.0,0.0,Consumer credit,-429,NaN,0.0,28,139.285714


In [80]:
x = bureau_full_df[['SK_ID_CURR', 'SK_ID_BUREAU']].groupby(by = ['SK_ID_CURR'])
x = x['SK_ID_BUREAU'].count().reset_index()
x = x.rename(index=str, columns={'SK_ID_BUREAU': 'BUREAU_NUMBER_OF_LOANS'})
bureau_full_df = bureau_full_df.merge(x, on=['SK_ID_CURR'], how='left')

In [81]:
bureau_full_df[bureau_full_df.SK_ID_CURR == 100001]

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,BUREAU_NUMBER_OF_LOANS
248484,100001,5896630,Closed,currency 1,-857,0,-492.0,-553.0,NaN,0,112500.0,0.0,0.0,0.0,Consumer credit,-155,0.0,7
248485,100001,5896631,Closed,currency 1,-909,0,-179.0,-877.0,NaN,0,279720.0,0.0,0.0,0.0,Consumer credit,-155,0.0,7
248486,100001,5896632,Closed,currency 1,-879,0,-514.0,-544.0,NaN,0,91620.0,0.0,0.0,0.0,Consumer credit,-155,0.0,7
248487,100001,5896633,Closed,currency 1,-1572,0,-1329.0,-1328.0,NaN,0,85500.0,0.0,0.0,0.0,Consumer credit,-155,0.0,7
248488,100001,5896634,Active,currency 1,-559,0,902.0,NaN,NaN,0,337680.0,113166.0,0.0,0.0,Consumer credit,-6,4630.5,7
248489,100001,5896635,Active,currency 1,-49,0,1778.0,NaN,NaN,0,378000.0,373239.0,0.0,0.0,Consumer credit,-16,10822.5,7
248490,100001,5896636,Active,currency 1,-320,0,411.0,NaN,NaN,0,168345.0,110281.5,NaN,0.0,Consumer credit,-10,9364.5,7


#### Gerar features baseadas no número de itens para cada cliente no CREDIT_ACTIVE

In [82]:
for credit_active_state in bureau_full_df['CREDIT_ACTIVE'].unique():
    x = bureau_full_df[bureau_full_df.CREDIT_ACTIVE == credit_active_state][['SK_ID_CURR', 'CREDIT_ACTIVE']].groupby(by = ['SK_ID_CURR'])
    x = x['CREDIT_ACTIVE'].count().reset_index()

    new_column_name = 'BUREAU_NUMBER_OF_%s_LOANS' % credit_active_state.upper().replace(' ', '_')
    x = x.rename(index=str, columns={'CREDIT_ACTIVE': new_column_name})
    bureau_full_df = bureau_full_df.merge(x, on=['SK_ID_CURR'], how='left')
    bureau_full_df[new_column_name] = bureau_full_df[new_column_name].fillna(0)

In [83]:
bureau_full_df[bureau_full_df.SK_ID_CURR == 100001]

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,BUREAU_NUMBER_OF_LOANS,BUREAU_NUMBER_OF_CLOSED_LOANS,BUREAU_NUMBER_OF_ACTIVE_LOANS,BUREAU_NUMBER_OF_SOLD_LOANS,BUREAU_NUMBER_OF_BAD DEBT_LOANS
248484,100001,5896630,Closed,currency 1,-857,0,-492.0,-553.0,NaN,0,112500.0,0.0,0.0,0.0,Consumer credit,-155,0.0,7,4.0,3.0,0.0,0.0
248485,100001,5896631,Closed,currency 1,-909,0,-179.0,-877.0,NaN,0,279720.0,0.0,0.0,0.0,Consumer credit,-155,0.0,7,4.0,3.0,0.0,0.0
248486,100001,5896632,Closed,currency 1,-879,0,-514.0,-544.0,NaN,0,91620.0,0.0,0.0,0.0,Consumer credit,-155,0.0,7,4.0,3.0,0.0,0.0
248487,100001,5896633,Closed,currency 1,-1572,0,-1329.0,-1328.0,NaN,0,85500.0,0.0,0.0,0.0,Consumer credit,-155,0.0,7,4.0,3.0,0.0,0.0
248488,100001,5896634,Active,currency 1,-559,0,902.0,NaN,NaN,0,337680.0,113166.0,0.0,0.0,Consumer credit,-6,4630.5,7,4.0,3.0,0.0,0.0
248489,100001,5896635,Active,currency 1,-49,0,1778.0,NaN,NaN,0,378000.0,373239.0,0.0,0.0,Consumer credit,-16,10822.5,7,4.0,3.0,0.0,0.0
248490,100001,5896636,Active,currency 1,-320,0,411.0,NaN,NaN,0,168345.0,110281.5,NaN,0.0,Consumer credit,-10,9364.5,7,4.0,3.0,0.0,0.0


#### Qual o tempo entre um emprestimo e outro?

In [84]:
grp = bureau_full_df[['SK_ID_CURR', 'DAYS_CREDIT']].groupby(by = ['SK_ID_CURR'])
grp1 = grp.apply(lambda x: x.sort_values(['DAYS_CREDIT'], ascending = False)).reset_index(drop = True)
grp1['DAYS_CREDIT1'] = grp1['DAYS_CREDIT'] * -1
grp1['DAYS_CREDIT_DIFF'] = grp1.groupby(by = ['SK_ID_CURR'])['DAYS_CREDIT1'].diff()
grp1['DAYS_CREDIT_DIFF'] = grp1['DAYS_CREDIT_DIFF'].fillna(0).astype('uint32')
del grp1['DAYS_CREDIT'], grp1['DAYS_CREDIT1']
grp1.head()

bureau_full_df = bureau_full_df.merge(grp1, on = ['SK_ID_CURR'], how = 'left')

In [85]:
# Qual o tempo medio entre os emprestimos realizados?
x = bureau_full_df[['SK_ID_CURR', 'DAYS_CREDIT_DIFF']].groupby(by = ['SK_ID_CURR'])['DAYS_CREDIT_DIFF'].mean()
x = x.to_frame().reset_index().rename(index=str, columns={'DAYS_CREDIT_DIFF': 'DAYS_CREDIT_DIFF_MEAN'})
bureau_full_df = bureau_full_df.merge(x, on=['SK_ID_CURR'], how='left')

In [86]:
bureau_full_df[bureau_full_df.SK_ID_CURR == 100002]

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,BUREAU_NUMBER_OF_LOANS,BUREAU_NUMBER_OF_CLOSED_LOANS,BUREAU_NUMBER_OF_ACTIVE_LOANS,BUREAU_NUMBER_OF_SOLD_LOANS,BUREAU_NUMBER_OF_BAD DEBT_LOANS,DAYS_CREDIT_DIFF,DAYS_CREDIT_DIFF_MEAN
6169431,100002,6158904,Closed,currency 1,-1125,0,-1038.0,-1038.0,NaN,0,40761.000,NaN,NaN,0.0,Credit card,-1038,0.0,8,6.0,2.0,0.0,0.0,0,166.75
6169432,100002,6158904,Closed,currency 1,-1125,0,-1038.0,-1038.0,NaN,0,40761.000,NaN,NaN,0.0,Credit card,-1038,0.0,8,6.0,2.0,0.0,0.0,373,166.75
6169433,100002,6158904,Closed,currency 1,-1125,0,-1038.0,-1038.0,NaN,0,40761.000,NaN,NaN,0.0,Credit card,-1038,0.0,8,6.0,2.0,0.0,0.0,169,166.75
6169434,100002,6158904,Closed,currency 1,-1125,0,-1038.0,-1038.0,NaN,0,40761.000,NaN,NaN,0.0,Credit card,-1038,0.0,8,6.0,2.0,0.0,0.0,397,166.75
6169435,100002,6158904,Closed,currency 1,-1125,0,-1038.0,-1038.0,NaN,0,40761.000,NaN,NaN,0.0,Credit card,-1038,0.0,8,6.0,2.0,0.0,0.0,1,166.75
6169436,100002,6158904,Closed,currency 1,-1125,0,-1038.0,-1038.0,NaN,0,40761.000,NaN,NaN,0.0,Credit card,-1038,0.0,8,6.0,2.0,0.0,0.0,78,166.75
6169437,100002,6158904,Closed,currency 1,-1125,0,-1038.0,-1038.0,NaN,0,40761.000,NaN,NaN,0.0,Credit card,-1038,0.0,8,6.0,2.0,0.0,0.0,4,166.75
6169438,100002,6158904,Closed,currency 1,-1125,0,-1038.0,-1038.0,NaN,0,40761.000,NaN,NaN,0.0,Credit card,-1038,0.0,8,6.0,2.0,0.0,0.0,312,166.75
6169439,100002,6158905,Closed,currency 1,-476,0,NaN,-48.0,NaN,0,0.000,0.0,NaN,0.0,Credit card,-47,NaN,8,6.0,2.0,0.0,0.0,0,166.75
6169440,100002,6158905,Closed,currency 1,-476,0,NaN,-48.0,NaN,0,0.000,0.0,NaN,0.0,Credit card,-47,NaN,8,6.0,2.0,0.0,0.0,373,166.75


#### Numero total de dias do emprestimo

In [87]:
x = bureau_full_df
x['DAYS_CREDIT_ENDDATE'] = x['DAYS_CREDIT_ENDDATE'].fillna(0)
x['DAYS_CREDIT_POSITIVE'] = x['DAYS_CREDIT'] * -1
bureau_full_df['TOTAL_OF_LOAN_DAYS'] = x['DAYS_CREDIT_POSITIVE'] + x['DAYS_CREDIT_ENDDATE']
bureau_full_df.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,BUREAU_NUMBER_OF_LOANS,BUREAU_NUMBER_OF_CLOSED_LOANS,BUREAU_NUMBER_OF_ACTIVE_LOANS,BUREAU_NUMBER_OF_SOLD_LOANS,BUREAU_NUMBER_OF_BAD DEBT_LOANS,DAYS_CREDIT_DIFF,DAYS_CREDIT_DIFF_MEAN,DAYS_CREDIT_POSITIVE,TOTAL_OF_LOAN_DAYS
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,11,5.0,6.0,0.0,0.0,0,166.272727,497,344.0
1,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,11,5.0,6.0,0.0,0.0,160,166.272727,497,344.0
2,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,11,5.0,6.0,0.0,0.0,0,166.272727,497,344.0
3,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,11,5.0,6.0,0.0,0.0,5,166.272727,497,344.0
4,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,11,5.0,6.0,0.0,0.0,65,166.272727,497,344.0


In [88]:
# Qual o tempo medio entre os emprestimos realizados?
x = bureau_full_df[['SK_ID_CURR', 'TOTAL_OF_LOAN_DAYS']].groupby(by = ['SK_ID_CURR'])['TOTAL_OF_LOAN_DAYS'].mean()
x = x.to_frame().reset_index().rename(index=str, columns={'TOTAL_OF_LOAN_DAYS': 'MEAN_OF_LOAN_DAYS'})
bureau_full_df = bureau_full_df.merge(x, on=['SK_ID_CURR'], how='left')
bureau_full_df.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,BUREAU_NUMBER_OF_LOANS,BUREAU_NUMBER_OF_CLOSED_LOANS,BUREAU_NUMBER_OF_ACTIVE_LOANS,BUREAU_NUMBER_OF_SOLD_LOANS,BUREAU_NUMBER_OF_BAD DEBT_LOANS,DAYS_CREDIT_DIFF,DAYS_CREDIT_DIFF_MEAN,DAYS_CREDIT_POSITIVE,TOTAL_OF_LOAN_DAYS,MEAN_OF_LOAN_DAYS
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,11,5.0,6.0,0.0,0.0,0,166.272727,497,344.0,3169.0
1,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,11,5.0,6.0,0.0,0.0,160,166.272727,497,344.0,3169.0
2,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,11,5.0,6.0,0.0,0.0,0,166.272727,497,344.0,3169.0
3,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,11,5.0,6.0,0.0,0.0,5,166.272727,497,344.0,3169.0
4,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,11,5.0,6.0,0.0,0.0,65,166.272727,497,344.0,3169.0


#### Existem dois ou mais pedidos de empréstimo ao mesmo tempo?

#### Existem créditos atrasados (CREDIT_DAY_OVERDUE)?

#### Gravando nova tabela em CSV para ser usada no modelo externo

In [89]:
application_train_df = pd.read_csv('dataset/application_train.csv')

In [90]:
features = ['SK_ID_CURR',  'BUREAU_NUMBER_OF_LOANS', 'BUREAU_NUMBER_OF_CLOSED_LOANS', 'BUREAU_NUMBER_OF_ACTIVE_LOANS', 'BUREAU_NUMBER_OF_SOLD_LOANS', 'DAYS_CREDIT_DIFF_MEAN', 'MEAN_OF_LOAN_DAYS']
b = bureau_full_df[features].drop_duplicates().reset_index()
b.to_csv('dataset/bureau_processed.csv', index=False)

In [92]:
x = application_train_df.merge(right=b, on=['SK_ID_CURR'], how='left')
x.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,index,BUREAU_NUMBER_OF_LOANS,BUREAU_NUMBER_OF_CLOSED_LOANS,BUREAU_NUMBER_OF_ACTIVE_LOANS,BUREAU_NUMBER_OF_SOLD_LOANS,DAYS_CREDIT_DIFF_MEAN,MEAN_OF_LOAN_DAYS
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,6169431.0,8.0,6.0,2.0,0.0,166.75,612.25
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0.0,0.0,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,9351490.0,4.0,3.0,1.0,0.0,495.00,856.25
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-

In [93]:
bureau_full_df[bureau_full_df.SK_ID_CURR == 100006]

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,BUREAU_NUMBER_OF_LOANS,BUREAU_NUMBER_OF_CLOSED_LOANS,BUREAU_NUMBER_OF_ACTIVE_LOANS,BUREAU_NUMBER_OF_SOLD_LOANS,BUREAU_NUMBER_OF_BAD DEBT_LOANS,DAYS_CREDIT_DIFF,DAYS_CREDIT_DIFF_MEAN,DAYS_CREDIT_POSITIVE,TOTAL_OF_LOAN_DAYS,MEAN_OF_LOAN_DAYS


In [59]:
application_train_df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0.0,0.0,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [55]:
bureau_full_df.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,BUREAU_NUMBER_OF_LOANS,BUREAU_NUMBER_OF_CLOSED_LOANS,BUREAU_NUMBER_OF_ACTIVE_LOANS,BUREAU_NUMBER_OF_SOLD_LOANS,DAYS_CREDIT_DIFF,DAYS_CREDIT_DIFF_MEAN,DAYS_CREDIT_POSITIVE,TOTAL_OF_LOAN_DAYS,MEAN_OF_LOAN_DAYS
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,7,1.0,6.0,0.0,0,83.714286,497,344.0,4606.0
1,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,7,1.0,6.0,0.0,160,83.714286,497,344.0,4606.0
2,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,7,1.0,6.0,0.0,0,83.714286,497,344.0,4606.0
3,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,7,1.0,6.0,0.0,5,83.714286,497,344.0,4606.0
4,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,7,1.0,6.0,0.0,65,83.714286,497,344.0,4606.0


In [18]:
features = ['SK_ID_CURR', 'BUREAU_NUMBER_OF_LOANS', 'BUREAU_NUMBER_OF_CLOSED_LOANS', 'BUREAU_NUMBER_OF_ACTIVE_LOANS', 'BUREAU_NUMBER_OF_SOLD_LOANS', 'BUREAU_NUMBER_OF_BAD DEBT_LOANS', 'DAYS_CREDIT_DIFF', 'DAYS_CREDIT_DIFF_MEAN', 'MEAN_OF_LOAN_DAYS']
bureau_full_df[features].to_csv('dataset/bureau_processed.csv', index=False)

### Criando um modelo baseado nesta tabela